# PyCity Schools Analysis

Looking at the school summary charts, the all charter schools out performed each district school. Looking specifically at Overall Passing Rate, the lowest performing charter school had a 94.38% passing rate versus the highest performing district school had a 74.31% passing rate.  When compared agains the district summary numbers, the charter schools are performing well above average in both math and reading. 

According to the "By Grade" scores, there is little difference (maybe 1.5-2%) between the average scores at each grade level within each individual school.  This may suggest, where scores are low across all grade levels, that certain schools may need to re-evaluate their entire curriculum as their may be systemtic issue in how these courses are currently being taught. 

More analysis would be warranted to confirm this theory but it appears that students attending small and medium size schools, that spend less than $611 per student, will have a very high chance, approx 93 percent, of passing.  Both small and medium size schools had an average passing rate that met or exceeding a 93 percent overall passing rate.  Schools spending less that $611 per student also realized an average passing rate at approximately 93%.

---

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
school_data = school_data.rename(columns={"name":"school_name"})
student_data = pd.read_csv(student_data_to_load)
student_data = student_data.rename(columns={"school":"school_name"})

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, on=["school_name", "school_name"])
school_summary_noformat_df = school_data_complete.copy()
#school_data_complete.head()

## District Summary

In [2]:
#District Summary
list_schools = school_data.drop_duplicates(subset='school_name')
total_schools = len(list_schools)
total_students = list_schools['size'].sum()
total_budget = list_schools['budget'].sum()

avg_reading = school_data_complete['reading_score'].mean()
avg_math = school_data_complete['math_score'].mean()
overall_passing_rate = ((avg_reading+avg_math)/2)
pass_reading = (len(school_data_complete[school_data_complete['reading_score'] >= 70])/total_students)*100
pass_math = (len(school_data_complete[school_data_complete['math_score'] >= 70])/total_students)*100

df_district_summary = pd.DataFrame([
    {
        "Total Schools":total_schools,
        "Total Students":total_students,
        "Total Budget":total_budget,
        "Average Math Score":avg_math,
        "Average Reading Score":avg_reading,
        "% Passing Math":pass_math,
        "% Passing Reading":pass_reading,
        "% Overall Passing Rate":overall_passing_rate
    }
])

df_district_organized = df_district_summary[["Total Schools","Total Students","Total Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]]
df_district_organized

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

In [3]:
#computer average reading score and percent passed reading
read_summary_df = school_data_complete[['school_name','type','size','budget','reading_score']]
pd.to_numeric(read_summary_df['reading_score'])
read_summary = read_summary_df.groupby(['school_name'])
read_summary_df = read_summary_df.set_index(['school_name'])
read_summary_df['Per Student Budget ($)'] = read_summary_df['budget']/read_summary_df['size']
read_summary_df['Avg Reading Score (%)'] = read_summary['reading_score'].mean()/100
read_summary_df = read_summary_df[read_summary_df['reading_score']>=70]
count_df = read_summary_df.groupby(['school_name']).count()
read_summary_df['Number Passed Reading'] = count_df['reading_score']
read_summary_df['Reading Passing Rate (%)'] = (read_summary_df['Number Passed Reading']/read_summary_df['size'])
del read_summary_df['reading_score']
del read_summary_df['Number Passed Reading']
read_summary_df = read_summary_df.drop_duplicates()
#read_summary_df.head()

In [4]:
#computer average math score and percent passed math
math_summary_df = school_data_complete[['school_name','type','size','budget','math_score']]
pd.to_numeric(math_summary_df['math_score'])
math_summary = math_summary_df.groupby(['school_name'])
math_summary_df = math_summary_df.set_index(['school_name'])
math_summary_df['budget']
math_summary_df['Per Student Budget ($)'] = math_summary_df['budget']/math_summary_df['size']
math_summary_df['Avg Math Score (%)'] = math_summary['math_score'].mean()/100
math_summary_df = math_summary_df[math_summary_df['math_score']>=70]
count_df = math_summary_df.groupby(['school_name']).count()
math_summary_df['Number Passed Math'] = count_df['math_score']
math_summary_df['Math Passing Rate (%)'] = (math_summary_df['Number Passed Math']/math_summary_df['size'])
del math_summary_df['math_score']
del math_summary_df['Number Passed Math']
math_summary_df = math_summary_df.drop_duplicates()
#math_summary_df.head()

In [5]:
#combine dataframes and calculate the overall passing rate.
school_summary_df = pd.merge(read_summary_df, math_summary_df, on=['school_name', 'type', 'size','budget', 'Per Student Budget ($)'])
school_summary_df = school_summary_df.rename(columns={'type':'School Type', 'size':'Student Population','budget':'School Budget ($)'})
school_summary_df['Overall Passing Rate (%)'] = (school_summary_df['Reading Passing Rate (%)'] + school_summary_df['Math Passing Rate (%)'])/2
#school_summary_df.head()


In [6]:
school_summary_df['School Budget ($)'] = school_summary_df['School Budget ($)'].map('${:,.2f}'.format)
school_summary_df['Per Student Budget ($)'] = school_summary_df['Per Student Budget ($)'].map('${:,.2f}'.format)
school_summary_df['Avg Reading Score (%)'] = school_summary_df['Avg Reading Score (%)'].map('{:.2%}'.format)
school_summary_df['Avg Math Score (%)'] = school_summary_df['Avg Math Score (%)'].map("{:.2%}".format)
school_summary_df['Reading Passing Rate (%)'] = school_summary_df['Reading Passing Rate (%)'].map('{0:.2%}'.format)
school_summary_df['Math Passing Rate (%)'] = school_summary_df['Math Passing Rate (%)'].map('{0:.2%}'.format)
school_summary_df['Overall Passing Rate (%)'] = school_summary_df['Overall Passing Rate (%)'].map('{0:.2%}'.format)
school_summary_df


,School Type,Student Population,School Budget ($),Per Student Budget ($),Avg Reading Score (%),Reading Passing Rate (%),Avg Math Score (%),Math Passing Rate (%),Overall Passing Rate (%)
school_name,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",$655.00,81.18%,81.32%,76.63%,65.68%,73.50%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.16%,80.74%,76.71%,65.99%,73.36%
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.73%,95.85%,83.36%,93.87%,94.86%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,80.93%,80.86%,77.29%,66.75%,73.81%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.82%,97.14%,83.35%,93.39%,95.27%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.99%,96.54%,83.27%,93.87%,95.20%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.98%,97.04%,83.06%,94.13%,95.59%
Bailey High School,District,4976,"$3,124,928.00",$628.00,81.03%,81.93%,77.05%,66.68%,74.31%
Holden High School,Charter,427,"$248,087.00",$581.00,83.81%,96.25%,83.80%,92.51%,94.38%


## Top Performing Schools (By Passing Rate)

In [7]:
school_best_df = school_summary_df.sort_values("Overall Passing Rate (%)", ascending=False)
school_best_df.head(15)

,School Type,Student Population,School Budget ($),Per Student Budget ($),Avg Reading Score (%),Reading Passing Rate (%),Avg Math Score (%),Math Passing Rate (%),Overall Passing Rate (%)
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.98%,97.04%,83.06%,94.13%,95.59%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.85%,97.31%,83.42%,93.27%,95.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.82%,97.14%,83.35%,93.39%,95.27%
Pena High School,Charter,962,"$585,858.00",$609.00,84.04%,95.95%,83.84%,94.59%,95.27%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.99%,96.54%,83.27%,93.87%,95.20%
Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.95%,96.61%,83.68%,93.33%,94.97%
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.73%,95.85%,83.36%,93.87%,94.86%
Holden High School,Charter,427,"$248,087.00",$581.00,83.81%,96.25%,83.80%,92.51%,94.38%
Bailey High School,District,4976,"$3,124,928.00",$628.00,81.03%,81.93%,77.05%,66.68%,74.31%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [8]:
school_worst_df = school_summary_df.sort_values("Overall Passing Rate (%)", ascending=True)
school_worst_df.head(5)

,School Type,Student Population,School Budget ($),Per Student Budget ($),Avg Reading Score (%),Reading Passing Rate (%),Avg Math Score (%),Math Passing Rate (%),Overall Passing Rate (%)
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,80.74%,80.22%,76.84%,66.37%,73.29%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.16%,80.74%,76.71%,65.99%,73.36%
Huang High School,District,2917,"$1,910,635.00",$655.00,81.18%,81.32%,76.63%,65.68%,73.50%
Johnson High School,District,4761,"$3,094,650.00",$650.00,80.97%,81.22%,77.07%,66.06%,73.64%
Ford High School,District,2739,"$1,763,916.00",$644.00,80.75%,79.30%,77.10%,68.31%,73.80%


## Math Scores by Grade

In [9]:
math_scores_df = school_data_complete[['grade', 'school_name','math_score', 'size']]
math_scores_df = math_scores_df.rename(columns={'school_name':'School Name', 'grade':'Grade','math_score':'Average Math Score (%)'})

math_scores_avg = (math_scores_df['Average Math Score (%)'].groupby([math_scores_df['School Name'], math_scores_df['Grade']])).mean()
math_scoresfinal_df = pd.DataFrame(math_scores_avg)
math_scoresfinal_df['Average Math Score (%)'] = math_scoresfinal_df['Average Math Score (%)'].map('{:.2f}'.format)
math_scoresfinal_df

Average Math Score (%)
School Name           Grade                       
Bailey High School    10th                   77.00
                      11th                   77.52
                      12th                   76.49
                      9th                    77.08
Cabrera High School   10th                   83.15
                      11th                   82.77
                      12th                   83.28
                      9th                    83.09
Figueroa High School  10th                   76.54
                      11th                   76.88
                      12th                   77.15
                      9th                    76.40
Ford High School      10th                   77.67
                      11th                   76.92
                      12th                   76.18
                      9th                    77.36
Griffin High School   10th                   84.23
                      11th                   83.84
                      12th                   83.36
                      9th                    82.04
Hernandez High School 10th                   77.34
                      11th                   77.14
                      12th                   77.19
                      9th                    77.44
Holden High School    10th                   83.43
                      11th                   85.00
                      12th                   82.86
                      9th                    83.79
Huang High School     10th                   75.91
                      11th                   76.45
                      12th                   77.23
                      9th                    77.03
Johnson High School   10th                   76.69
                      11th                   77.49
                      12th                   76.86
                      9th                    77.19
Pena High School      10th                   83.37
                      11th                   84.33
                      12th                   84.12
                      9th                    83.63
Rodriguez High School 10th                   76.61
                      11th                   76.40
                      12th                   77.69
                      9th                    76.86
Shelton High School   10th                   82.92
                      11th                   83.38
                      12th                   83.78
                      9th                    83.42
Thomas High School    10th                   83.09
                      11th                   83.50
                      12th                   83.50
                      9th                    83.59
Wilson High School    10th                   83.72
                      11th                   83.20
                      12th                   83.04
                      9th                    83.09
Wright High School    10th                   84.01
                      11th                   83.84
                      12th                   83.64
                      9th                    83.26

## Reading Score by Grade 

In [10]:
reading_scores_df = school_data_complete[['grade', 'school_name','reading_score', 'size']]
reading_scores_df = reading_scores_df.rename(columns={'school_name':'School Name', 'grade':'Grade','reading_score':'Average Reading Score (%)'})

reading_scores_avg = (reading_scores_df['Average Reading Score (%)'].groupby([reading_scores_df['School Name'], reading_scores_df['Grade']])).mean()
reading_scoresfinal_df = pd.DataFrame(reading_scores_avg)
reading_scoresfinal_df['Average Reading Score (%)'] = reading_scoresfinal_df['Average Reading Score (%)'].map('{:.2f}'.format)
reading_scoresfinal_df

Average Reading Score (%)
School Name           Grade                          
Bailey High School    10th                      80.91
                      11th                      80.95
                      12th                      80.91
                      9th                       81.30
Cabrera High School   10th                      84.25
                      11th                      83.79
                      12th                      84.29
                      9th                       83.68
Figueroa High School  10th                      81.41
                      11th                      80.64
                      12th                      81.38
                      9th                       81.20
Ford High School      10th                      81.26
                      11th                      80.40
                      12th                      80.66
                      9th                       80.63
Griffin High School   10th                      83.71
                      11th                      84.29
                      12th                      84.01
                      9th                       83.37
Hernandez High School 10th                      80.66
                      11th                      81.40
                      12th                      80.86
                      9th                       80.87
Holden High School    10th                      83.32
                      11th                      83.82
                      12th                      84.70
                      9th                       83.68
Huang High School     10th                      81.51
                      11th                      81.42
                      12th                      80.31
                      9th                       81.29
Johnson High School   10th                      80.77
                      11th                      80.62
                      12th                      81.23
                      9th                       81.26
Pena High School      10th                      83.61
                      11th                      84.34
                      12th                      84.59
                      9th                       83.81
Rodriguez High School 10th                      80.63
                      11th                      80.86
                      12th                      80.38
                      9th                       80.99
Shelton High School   10th                      83.44
                      11th                      84.37
                      12th                      82.78
                      9th                       84.12
Thomas High School    10th                      84.25
                      11th                      83.59
                      12th                      83.83
                      9th                       83.73
Wilson High School    10th                      84.02
                      11th                      83.76
                      12th                      84.32
                      9th                       83.94
Wright High School    10th                      83.81
                      11th                      84.16
                      12th                      84.07
                      9th                       83.83

## Scores by School Spending

In [19]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 590, 611, 633, 655]
group_names = ["<$590", "$590-611", "$611-633", "$633-655"]

In [20]:
school_spending_df = school_summary_noformat_df[['Student ID','school_name', 'reading_score','math_score','type','size','budget']]
school_spending_df['Per Student Budget ($)'] = school_spending_df['budget']/school_spending_df['size']
school_spending_df['Spending_Range'] = pd.cut(school_spending_df['Per Student Budget ($)'], spending_bins, labels=group_names)
school_spending_df.dtypes
school_spending_grouped = school_spending_df.groupby('Spending_Range')
school_spending_df = school_spending_df.set_index(['Spending_Range'])
total_count = school_spending_grouped['school_name'].count()
school_spending_df['Bin_Size'] = total_count

school_spending_df['Avg Reading Score (%)'] = school_spending_grouped['reading_score'].mean()/100
school_spending_df = school_spending_df[school_spending_df['reading_score']>=70]
read_count_passed = school_spending_df.groupby('Spending_Range').count()
school_spending_df['Number Passed Reading'] = read_count_passed['reading_score']
school_spending_df['Reading Passing Rate (%)'] = (school_spending_df['Number Passed Reading']/school_spending_df['Bin_Size'])

school_spending_df['Avg Math Score (%)'] = school_spending_grouped['math_score'].mean()/100
school_spending_df = school_spending_df[school_spending_df['math_score']>=70]

math_count_passed = school_spending_df.groupby('Spending_Range').count()
school_spending_df['Number Passed Math'] = math_count_passed['math_score']
school_spending_df['Math Passing Rate (%)'] = (school_spending_df['Number Passed Math']/school_spending_df['Bin_Size'])

school_spending_df['Overall Passing Rate (%)'] = (school_spending_df['Math Passing Rate (%)'] + school_spending_df['Reading Passing Rate (%)'])/2

school_spending_final = school_spending_df[['Avg Reading Score (%)','Avg Math Score (%)','Reading Passing Rate (%)','Math Passing Rate (%)', 'Overall Passing Rate (%)']]
school_spending_final = school_spending_final.drop_duplicates()
school_spending_final['Avg Reading Score (%)'] = school_spending_final['Avg Reading Score (%)'].map('{:.2%}'.format)
school_spending_final['Avg Math Score (%)'] = school_spending_final['Avg Math Score (%)'].map('{:.2%}'.format)
school_spending_final['Reading Passing Rate (%)'] = school_spending_final['Reading Passing Rate (%)'].map('{:.2%}'.format)
school_spending_final['Math Passing Rate (%)'] = school_spending_final['Math Passing Rate (%)'].map('{:.2%}'.format)
school_spending_final['Overall Passing Rate (%)'] = school_spending_final['Overall Passing Rate (%)'].map('{:.2%}'.format)

school_spending_final.sort_values('Spending_Range').head()

C:\Users\goodmanclint\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\goodmanclint\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Avg Reading Score (%),Avg Math Score (%),Reading Passing Rate (%),Math Passing Rate (%),Overall Passing Rate (%)
Spending_Range,,,,,
<$590,83.96%,83.36%,96.69%,90.64%,93.66%
$590-611,83.84%,83.53%,95.89%,90.12%,93.00%
$611-633,81.67%,78.48%,85.40%,62.83%,74.12%
$633-655,81.15%,77.42%,81.82%,56.07%,68.95%


## Scores by School Size

In [13]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [14]:
school_size_df = school_summary_noformat_df[['school_name', 'reading_score','math_score','type','size','budget']]
pd.to_numeric(school_size_df['size'])
school_size_df['School Size Range'] = pd.cut(school_size_df['size'], size_bins, labels=group_names)
school_size_df.dtypes
school_size_grouped = school_size_df.groupby('School Size Range')
school_size_df = school_size_df.set_index(['School Size Range'])
total_count = school_size_grouped['school_name'].count()
school_size_df['Bin_Size'] = total_count

school_size_df['Avg Reading Score (%)'] = school_size_grouped['reading_score'].mean()/100
school_size_df = school_size_df[school_size_df['reading_score']>=70]
read_count_passed = school_size_df.groupby('School Size Range').count()
school_size_df['Number Passed Reading'] = read_count_passed['reading_score']
school_size_df['Reading Passing Rate (%)'] = (school_size_df['Number Passed Reading']/school_size_df['Bin_Size'])

school_size_df['Avg Math Score (%)'] = school_size_grouped['math_score'].mean()/100
school_size_df = school_size_df[school_size_df['math_score']>=70]

math_count_passed = school_size_df.groupby('School Size Range').count()
school_size_df['Number Passed Math'] = math_count_passed['math_score']
school_size_df['Math Passing Rate (%)'] = (school_size_df['Number Passed Math']/school_size_df['Bin_Size'])

school_size_df['Overall Passing Rate (%)'] = (school_size_df['Math Passing Rate (%)'] + school_size_df['Reading Passing Rate (%)'])/2

school_size_final = school_size_df[['Avg Reading Score (%)','Avg Math Score (%)','Reading Passing Rate (%)','Math Passing Rate (%)', 'Overall Passing Rate (%)']]
school_size_final = school_size_final.drop_duplicates()
school_size_final['Avg Reading Score (%)'] = school_size_final['Avg Reading Score (%)'].map('{:.2%}'.format)
school_size_final['Avg Math Score (%)'] = school_size_final['Avg Math Score (%)'].map('{:.2%}'.format)
school_size_final['Reading Passing Rate (%)'] = school_size_final['Reading Passing Rate (%)'].map('{:.2%}'.format)
school_size_final['Math Passing Rate (%)'] = school_size_final['Math Passing Rate (%)'].map('{:.2%}'.format)
school_size_final['Overall Passing Rate (%)'] = school_size_final['Overall Passing Rate (%)'].map('{:.2%}'.format)

school_size_final.sort_values('School Size Range').head()

C:\Users\goodmanclint\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Avg Reading Score (%),Avg Math Score (%),Reading Passing Rate (%),Math Passing Rate (%),Overall Passing Rate (%)
School Size Range,,,,,
Small (<1000),83.97%,83.83%,96.04%,90.14%,93.09%
Medium (1000-2000),83.87%,83.37%,96.77%,90.62%,93.70%
Large (2000-5000),81.20%,77.48%,82.13%,56.57%,69.35%


## Scores by School Type

In [15]:
school_type_df = school_summary_noformat_df[['school_name', 'reading_score','math_score','type','size','budget']]
school_type_df = school_type_df.rename(columns={'type':'School Type'})
school_type_df.dtypes
school_type_grouped = school_type_df.groupby('School Type')
school_type_df = school_type_df.set_index(['School Type'])
total_count = school_type_grouped['school_name'].count()
school_type_df['Bin_Size'] = total_count

school_type_df['Avg Reading Score (%)'] = school_type_grouped['reading_score'].mean()/100
school_type_df = school_type_df[school_type_df['reading_score']>=70]
read_count_passed = school_type_df.groupby('School Type').count()
school_type_df['Number Passed Reading'] = read_count_passed['reading_score']
school_type_df['Reading Passing Rate (%)'] = (school_type_df['Number Passed Reading']/school_type_df['Bin_Size'])

school_type_df['Avg Math Score (%)'] = school_type_grouped['math_score'].mean()/100
school_type_df = school_type_df[school_type_df['math_score']>=70]
math_count_passed = school_type_df.groupby('School Type').count()
school_type_df['Number Passed Math'] = math_count_passed['math_score']
school_type_df['Math Passing Rate (%)'] = (school_type_df['Number Passed Math']/school_type_df['Bin_Size'])

school_type_df['Overall Passing Rate (%)'] = (school_type_df['Math Passing Rate (%)'] + school_type_df['Reading Passing Rate (%)'])/2

school_type_final = school_type_df[['Avg Reading Score (%)','Avg Math Score (%)','Reading Passing Rate (%)','Math Passing Rate (%)', 'Overall Passing Rate (%)']]
school_type_final = school_type_final.drop_duplicates()
school_type_final['Avg Reading Score (%)'] = school_type_final['Avg Reading Score (%)'].map('{:.2%}'.format)
school_type_final['Avg Math Score (%)'] = school_type_final['Avg Math Score (%)'].map('{:.2%}'.format)
school_type_final['Reading Passing Rate (%)'] = school_type_final['Reading Passing Rate (%)'].map('{:.2%}'.format)
school_type_final['Math Passing Rate (%)'] = school_type_final['Math Passing Rate (%)'].map('{:.2%}'.format)
school_type_final['Overall Passing Rate (%)'] = school_type_final['Overall Passing Rate (%)'].map('{:.2%}'.format)

school_type_final.head()

,Avg Reading Score (%),Avg Math Score (%),Reading Passing Rate (%),Math Passing Rate (%),Overall Passing Rate (%)
School Type,,,,,
District,80.96%,76.99%,80.91%,53.70%,67.30%
Charter,83.90%,83.41%,96.65%,90.56%,93.60%
